In [1]:
import time
import warnings
import sys
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectPercentile, f_classif, SelectFromModel
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_recall_curve, auc, mean_squared_error, \
    r2_score, mean_absolute_error,cohen_kappa_score,accuracy_score,f1_score,matthews_corrcoef,precision_score,recall_score
from sklearn.model_selection import train_test_split
import multiprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor

start = time.time()
warnings.filterwarnings("ignore")

def standardize(col):
    return (col - np.mean(col)) / np.std(col)

# the metrics for classification
def statistical(y_true, y_pred, y_pro):
    c_mat = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = list(c_mat.flatten())
    se = tp / (tp + fn)
    sp = tn / (tn + fp)
    auc_prc = auc(precision_recall_curve(y_true, y_pro, pos_label=1)[1],
                  precision_recall_curve(y_true, y_pro, pos_label=1)[0])
    acc = (tp + tn) / (tn + fp + fn + tp)
#     acc_skl = accuracy_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pro)
    recall = se
#     recall_skl = recall_score(y_true, y_pred)
    precision = tp / (tp + fp)
#     precision_skl = precision_score(y_true, y_pred)
    f1 = 2 * (precision * recall) / (precision + recall) # F1 = 2 * (precision * recall) / (precision + recall)
#     f1_skl = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true,y_pred)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn) + 1e-8)
#     mcc_skl = matthews_corrcoef(y_true,y_pred)
    return tn,fp,fn,tp,se,sp,auc_prc,acc,auc_roc,recall,precision,f1,kappa,mcc

def all_one_zeros(data):
    if (len(np.unique(data)) == 2):
        flag = False
    else:
        flag = True
    return flag


feature_selection = False

In [2]:
tasks_dic = {'1-AR-AlvaSlim-DMPNN-6108-Normalize-group.csv': ['activity']}
file_name = '1-AR-AlvaSlim-DMPNN-6108-Normalize-group.csv'
task_type = 'cla'  # 'reg' or 'cla'
dataset_label = file_name.split('/')[-1].split('_')[0]
tasks = tasks_dic[dataset_label]
OPT_ITERS = 50
repetitions = 10
num_pools = 10
unbalance = True
patience = 100
ecfp = True
space_ = {'num_leaves': hp.choice('num_leaves', range(2,256,1)),
          'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
          'n_estimators': hp.choice('n_estimators', range(10,300,1)),
          'min_split_gain': hp.uniform('min_split_gain', 0., 1),
          'reg_alpha': hp.loguniform('reg_alpha', 1e-10, 10.0),
          'reg_lambda': hp.loguniform('reg_lambda', 1e-10, 10.0),
          'feature_fraction': hp.uniform('feature_fraction', 0.4, 1),
          'bagging_fraction': hp.uniform('bagging_fraction', 0.4, 1),
          'bagging_freq': hp.choice('bagging_freq', range(0,7,1)),
          'min_child_samples': hp.choice('min_child_samples', range(1,100,1))
          }
num_leaves_ls = range(2,256,1)
n_estimators_ls = range(10,300,1)
bagging_freq_ls = range(0,7,1)
dataset = pd.read_csv(file_name)
pd_res = []

In [3]:
def hyper_runing(subtask):
    cols = [subtask]
    cols.extend(dataset.columns[(len(tasks) + 1):])
    sub_dataset = dataset[cols]

#     # detect the na in the subtask (y cloumn)
#     rm_index = sub_dataset[subtask][sub_dataset[subtask].isnull()].index
#     sub_dataset.drop(index=rm_index, inplace=True)

#     # remove the features with na
#     sub_dataset = sub_dataset.dropna(axis=1)
#     # *******************
#     # demension reduction
#     # *******************
#     # Removing features with low variance
#     # threshold = 0.05
#     data_fea_var = sub_dataset.iloc[:, 2:].var()
#     del_fea1 = list(data_fea_var[data_fea_var <= 0.05].index)
#     sub_dataset.drop(columns=del_fea1, inplace=True)

#     # pair correlations
#     # threshold = 0.95
#     data_fea_corr = sub_dataset.iloc[:, 2:].corr()
#     del_fea2_col = []
#     del_fea2_ind = []
#     length = data_fea_corr.shape[1]
#     for i in range(length):
#         for j in range(i + 1, length):
#             if abs(data_fea_corr.iloc[i, j]) >= 0.95:
#                 del_fea2_col.append(data_fea_corr.columns[i])
#                 del_fea2_ind.append(data_fea_corr.index[j])
#     sub_dataset.drop(columns=del_fea2_ind, inplace=True)

#     # standardize the features
#     cols_ = list(sub_dataset.columns)[2:]
#     if not ecfp :
#         sub_dataset[cols_] = sub_dataset[cols_].apply(standardize, axis=0)

    # get the attentivefp data splits
    data_tr = sub_dataset[sub_dataset['group'] == 'train']
    data_va = sub_dataset[sub_dataset['group'] == 'valid']
    data_te = sub_dataset[sub_dataset['group'] == 'test']

    # prepare data for training
    # training set
    data_tr_y = data_tr[subtask].values.reshape(-1, 1)
    data_tr_x = np.array(data_tr.iloc[:, 2:].values)

    # validation set
    data_va_y = data_va[subtask].values.reshape(-1, 1)
    data_va_x = np.array(data_va.iloc[:, 2:].values)

    # test set
    data_te_y = data_te[subtask].values.reshape(-1, 1)
    data_te_x = np.array(data_te.iloc[:, 2:].values)

    if feature_selection:
        # univariate feature selection
        trans1 = SelectPercentile(f_classif, percentile=80)
        trans1.fit(data_tr_x, data_tr_y)
        data_tr_x = trans1.transform(data_tr_x)
        data_va_x = trans1.transform(data_va_x)
        data_te_x = trans1.transform(data_te_x)

        # select from model
        clf = LGBMClassifier(n_jobs=12, random_state=1)
        clf = clf.fit(data_tr_x, data_tr_y)
        trans2 = SelectFromModel(clf, prefit=True)

        data_tr_x = trans2.transform(data_tr_x)
        data_va_x = trans2.transform(data_va_x)
        data_te_x = trans2.transform(data_te_x)

    num_fea = data_tr_x.shape[1]
    print('the num of retained features for the ' + dataset_label + ' ' + subtask + ' is:', num_fea)

    def hyper_opt(args):
        model = LGBMClassifier(**args, n_jobs=-1, random_state=1) if task_type == 'cla' else LGBMRegressor(**args, n_jobs=6,
                                                                                                   random_state=1)

        model.fit(data_tr_x, data_tr_y, eval_metric='auc' if task_type == 'cla' else 'rmse',
                  eval_set=[(data_va_x, data_va_y)],
                  early_stopping_rounds=patience, verbose=False)
        val_preds = model.predict_proba(data_va_x) if task_type == 'cla' else \
            model.predict(data_va_x)
        loss = 1 - roc_auc_score(data_va_y, val_preds[:, 1]) if task_type == 'cla' else np.sqrt(
            mean_squared_error(data_va_y, val_preds))
        return {'loss': loss, 'status': STATUS_OK}

    # start hyper-parameters optimization
    trials = Trials()
    best_results = fmin(hyper_opt, space_, algo=tpe.suggest, max_evals=OPT_ITERS, trials=trials, show_progressbar=False)
    print('the best hyper-parameters for ' + dataset_label + ' ' + subtask + ' are:  ', best_results)

    best_model = LGBMClassifier(num_leaves=num_leaves_ls[best_results['num_leaves']],
                                learning_rate=best_results['learning_rate'],
                                n_estimators=n_estimators_ls[best_results['n_estimators']],
                                min_split_gain=best_results['min_split_gain'],
                                reg_alpha=best_results['reg_alpha'],
                                reg_lambda=best_results['reg_lambda'],
                                feature_fraction=best_results['feature_fraction'],
                                bagging_fraction=best_results['bagging_fraction'],
                                bagging_freq=bagging_freq_ls[best_results['bagging_freq']],
                                min_child_samples=best_results['min_child_samples'],
                                n_jobs=-1, random_state=1) \
        if task_type == 'cla' else LGBMRegressor(
        num_leaves=num_leaves_ls[best_results['num_leaves']],
                                        learning_rate=best_results['learning_rate'],
                                        n_estimators=n_estimators_ls[best_results['n_estimators']],
                                        min_split_gain=best_results['min_split_gain'],
                                        reg_alpha=best_results['reg_alpha'],
                                        reg_lambda=best_results['reg_lambda'],
                                        feature_fraction=best_results['feature_fraction'],
                                        bagging_fraction=best_results['bagging_fraction'],
                                        bagging_freq=bagging_freq_ls[best_results['bagging_freq']],
                                        min_child_samples=best_results['min_child_samples'],
                                        n_jobs=-1, random_state=1, verbose=-1)  
    
    best_model.fit(data_tr_x, data_tr_y, eval_metric='auc' if task_type == 'cla' else 'rmse',
                   eval_set=[(data_va_x, data_va_y)],
                   early_stopping_rounds=patience, verbose=False)
    num_of_compounds = len(sub_dataset)

    if task_type == 'cla':
        # training set
        tr_pred = best_model.predict_proba(data_tr_x)
        tr_results = [dataset_label, subtask, 'tr', num_fea, num_of_compounds, data_tr_y[data_tr_y == 1].shape[0],
                      data_tr_y[data_tr_y == 0].shape[0],
                      data_tr_y[data_tr_y == 0].shape[0] / data_tr_y[data_tr_y == 1].shape[0],
                      num_leaves_ls[best_results['num_leaves']],
                      best_results['learning_rate'],
                      n_estimators_ls[best_results['n_estimators']],
                      best_results['min_split_gain'],
                      best_results['reg_alpha'],
                      best_results['reg_lambda'],
                      best_results['feature_fraction'],
                      best_results['bagging_fraction'],
                      bagging_freq_ls[best_results['bagging_freq']],
                      best_results['min_child_samples']
                      ]
        tr_results.extend(statistical(data_tr_y, np.argmax(tr_pred, axis=1), tr_pred[:, 1]))
        # validation set
        va_pred = best_model.predict_proba(data_va_x)
                      
        va_results = [dataset_label, subtask, 'va', num_fea, num_of_compounds, data_va_y[data_va_y == 1].shape[0],
                      data_va_y[data_va_y == 0].shape[0],
                      data_va_y[data_va_y == 0].shape[0] / data_va_y[data_va_y == 1].shape[0],
                      num_leaves_ls[best_results['num_leaves']],
                      best_results['learning_rate'],
                      n_estimators_ls[best_results['n_estimators']],
                      best_results['min_split_gain'],
                      best_results['reg_alpha'],
                      best_results['reg_lambda'],
                      best_results['feature_fraction'],
                      best_results['bagging_fraction'],
                      bagging_freq_ls[best_results['bagging_freq']],
                      best_results['min_child_samples']]
        va_results.extend(statistical(data_va_y, np.argmax(va_pred, axis=1), va_pred[:, 1]))

        # test set
        te_pred = best_model.predict_proba(data_te_x)
        te_results = [dataset_label, subtask, 'te', num_fea, num_of_compounds, data_te_y[data_te_y == 1].shape[0],
                      data_te_y[data_te_y == 0].shape[0],
                      data_te_y[data_te_y == 0].shape[0] / data_te_y[data_te_y == 1].shape[0],
                      num_leaves_ls[best_results['num_leaves']],
                      best_results['learning_rate'],
                      n_estimators_ls[best_results['n_estimators']],
                      best_results['min_split_gain'],
                      best_results['reg_alpha'],
                      best_results['reg_lambda'],
                      best_results['feature_fraction'],
                      best_results['bagging_fraction'],
                      bagging_freq_ls[best_results['bagging_freq']],
                      best_results['min_child_samples']]
        te_results.extend(statistical(data_te_y, np.argmax(te_pred, axis=1), te_pred[:, 1]))
    else:
        # training set
        tr_pred = best_model.predict(data_tr_x)
        tr_results = [dataset_label, subtask, 'tr', num_fea, num_of_compounds,
                      num_leaves_ls[best_results['num_leaves']],
                      best_results['learning_rate'],
                      n_estimators_ls[best_results['n_estimators']],
                      best_results['min_split_gain'],
                      best_results['reg_alpha'],
                      best_results['reg_lambda'],
                      best_results['feature_fraction'],
                      best_results['bagging_fraction'],
                      bagging_freq_ls[best_results['bagging_freq']],
                      best_results['min_child_samples'],
                      np.sqrt(mean_squared_error(data_tr_y, tr_pred)), r2_score(data_tr_y, tr_pred),
                      mean_absolute_error(data_tr_y, tr_pred)]

        # validation set
        va_pred = best_model.predict(data_va_x)
        va_results = [dataset_label, subtask, 'va', num_fea, num_of_compounds,
                      num_leaves_ls[best_results['num_leaves']],
                      best_results['learning_rate'],
                      n_estimators_ls[best_results['n_estimators']],
                      best_results['min_split_gain'],
                      best_results['reg_alpha'],
                      best_results['reg_lambda'],
                      best_results['feature_fraction'],
                      best_results['bagging_fraction'],
                      bagging_freq_ls[best_results['bagging_freq']],
                      best_results['min_child_samples'],
                      np.sqrt(mean_squared_error(data_va_y, va_pred)), r2_score(data_va_y, va_pred),
                      mean_absolute_error(data_va_y, va_pred)]

        # test set
        te_pred = best_model.predict(data_te_x)
        te_results = [dataset_label, subtask, 'te', num_fea, num_of_compounds,
                      num_leaves_ls[best_results['num_leaves']],
                      best_results['learning_rate'],
                      n_estimators_ls[best_results['n_estimators']],
                      best_results['min_split_gain'],
                      best_results['reg_alpha'],
                      best_results['reg_lambda'],
                      best_results['feature_fraction'],
                      best_results['bagging_fraction'],
                      bagging_freq_ls[best_results['bagging_freq']],
                      best_results['min_child_samples'],
                      np.sqrt(mean_squared_error(data_te_y, te_pred)), r2_score(data_te_y, te_pred),
                      mean_absolute_error(data_te_y, te_pred)]
    return tr_results, va_results, te_results


pool = multiprocessing.Pool(num_pools)
res = pool.starmap(hyper_runing, zip(tasks))
pool.close()
pool.join()
for item in res:
    for i in range(3):
        pd_res.append(item[i])
if task_type == 'cla':                    
    best_hyper = pd.DataFrame(pd_res, columns=['dataset', 'subtask', 'set',
                                               'num_of_retained_feature',
                                               'num_of_compounds', 'postives',
                                               'negtives', 'negtives/postives',
                                               'num_leaves', 
                                               'learning_rate','n_estimators','min_split_gain','reg_alpha',
                                               'reg_lambda','feature_fraction','bagging_fraction','bagging_freq','min_child_samples',
                                               'tn', 'fp', 'fn', 'tp', 'se', 'sp',
                                               'auc_prc', 'acc', 'auc_roc','recall','precision','f1','kappa','mcc'])
else:
    best_hyper = pd.DataFrame(pd_res, columns=['dataset', 'subtask', 'set',
                                               'num_leaves', 
                                               'learning_rate','n_estimators','min_split_gain','reg_alpha',
                                               'reg_lambda','feature_fraction','bagging_fraction','bagging_freq','min_child_samples',
                                               'rmse', 'r2', 'mae'])
best_hyper.to_csv('./model/' + dataset_label + '_LGB_hyperopt_info.csv', index=0)

if task_type == 'cla':
    print('train', best_hyper[best_hyper['set'] == 'tr']['auc_roc'].mean(),
          best_hyper[best_hyper['set'] == 'tr']['auc_prc'].mean())
    print('valid', best_hyper[best_hyper['set'] == 'va']['auc_roc'].mean(),
          best_hyper[best_hyper['set'] == 'va']['auc_prc'].mean())
    print('test', best_hyper[best_hyper['set'] == 'te']['auc_roc'].mean(),
          best_hyper[best_hyper['set'] == 'te']['auc_prc'].mean())
else:
    print('train', best_hyper[best_hyper['set'] == 'tr']['rmse'].mean(),
          best_hyper[best_hyper['set'] == 'tr']['r2'].mean(), best_hyper[best_hyper['set'] == 'tr']['mae'].mean())
    print('valid', best_hyper[best_hyper['set'] == 'va']['rmse'].mean(),
          best_hyper[best_hyper['set'] == 'va']['r2'].mean(), best_hyper[best_hyper['set'] == 'va']['mae'].mean())
    print('test', best_hyper[best_hyper['set'] == 'te']['rmse'].mean(),
          best_hyper[best_hyper['set'] == 'te']['r2'].mean(), best_hyper[best_hyper['set'] == 'te']['mae'].mean())
    

the num of retained features for the 1-AR-AlvaSlim-DMPNN-6108-Normalize-group.csv activity is: 1808
[LightGBM] [Warning] bagging_fraction is set=0.9674006030826344, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9674006030826344
[LightGBM] [Warning] feature_fraction is set=0.7139382314805469, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7139382314805469
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] bagging_fraction is set=0.7528658750223002, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7528658750223002
[LightGBM] [Warning] feature_fraction is set=0.5085510592403634, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5085510592403634
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] bagging_fraction is set=0.8452486855027967, subsample=1.0 wil

[LightGBM] [Warning] bagging_fraction is set=0.642789969587692, subsample=1.0 will be ignored. Current value: bagging_fraction=0.642789969587692
[LightGBM] [Warning] feature_fraction is set=0.9913251651821732, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9913251651821732
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] bagging_fraction is set=0.8549476406890033, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8549476406890033
[LightGBM] [Warning] feature_fraction is set=0.8733128141360251, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8733128141360251
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] bagging_fraction is set=0.825115638846915, subsample=1.0 will be ignored. Current value: bagging_fraction=0.825115638846915
[LightGBM] [Warning] feature_fraction i

[LightGBM] [Warning] feature_fraction is set=0.9641472595232651, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9641472595232651
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] bagging_fraction is set=0.45177877519490595, subsample=1.0 will be ignored. Current value: bagging_fraction=0.45177877519490595
[LightGBM] [Warning] feature_fraction is set=0.9961926571048102, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9961926571048102
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] bagging_fraction is set=0.9704925723735269, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9704925723735269
[LightGBM] [Warning] feature_fraction is set=0.7827872019507204, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7827872019507204
[LightGBM] [Warning] baggi

In [4]:
# 10 repetitions based on thr best hypers
dataset.drop(columns=['group'], inplace=True)
pd_res = []

In [5]:
# best_hyper = pd.read_csv('model/1-AR-AlvaSlim-DMPNN-6108-Normalize-group.csv_LGB_hyperopt_info.csv')

In [6]:
def best_model_runing(split):
    seed = split
    if task_type == 'cla':
        while True:
            training_data, data_te = train_test_split(sub_dataset, test_size=0.1, random_state=seed)
            # the training set was further splited into the training set and validation set
            data_tr, data_va = train_test_split(training_data, test_size=0.1, random_state=seed)
            if (all_one_zeros(data_tr[subtask]) or all_one_zeros(data_va[subtask]) or all_one_zeros(data_te[subtask])):
                print(
                    '\ninvalid random seed {} due to one class presented in the {} splitted sets...'.format(seed,
                                                                                                            subtask))
                print('Changing to another random seed...\n')
                seed = np.random.randint(50, 999999)
            else:
                print('random seed used in repetition {} is {}'.format(split, seed))
                break
    else:
        training_data, data_te = train_test_split(sub_dataset, test_size=0.1, random_state=seed)
        # the training set was further splited into the training set and validation set
        data_tr, data_va = train_test_split(training_data, test_size=0.1, random_state=seed)

    # prepare data for training
    # training set
    data_tr_y = data_tr[subtask].values.reshape(-1, 1)
    data_tr_x = np.array(data_tr.iloc[:, 1:].values)

    # validation set
    data_va_y = data_va[subtask].values.reshape(-1, 1)
    data_va_x = np.array(data_va.iloc[:, 1:].values)

    # test set
    data_te_y = data_te[subtask].values.reshape(-1, 1)
    data_te_x = np.array(data_te.iloc[:, 1:].values)
          
    num_fea = data_tr_x.shape[1]
    pos_weight = (len(sub_dataset) - sum(sub_dataset[subtask])) / sum(sub_dataset[subtask])
    model = LGBMClassifier(
                          num_leaves=best_hyper[best_hyper.subtask == subtask].iloc[0,]['num_leaves'],
                          learning_rate=best_hyper[best_hyper.subtask == subtask].iloc[0,]['learning_rate'],
                          n_estimators=best_hyper[best_hyper.subtask == subtask].iloc[0,]['n_estimators'],
                          min_split_gain=best_hyper[best_hyper.subtask == subtask].iloc[0,]['min_split_gain'],
                          reg_alpha=best_hyper[best_hyper.subtask == subtask].iloc[0,]['reg_alpha'],
                          reg_lambda=best_hyper[best_hyper.subtask == subtask].iloc[0,]['reg_lambda'],
                          feature_fraction=best_hyper[best_hyper.subtask == subtask].iloc[0,]['feature_fraction'],
                          bagging_fraction=best_hyper[best_hyper.subtask == subtask].iloc[0,]['bagging_fraction'],
                          bagging_freq=best_hyper[best_hyper.subtask == subtask].iloc[0,]['bagging_freq'],
                          min_child_samples=best_hyper[best_hyper.subtask == subtask].iloc[0,]['min_child_samples'],
                          n_jobs=-1, random_state=1) \
        if task_type == 'cla' else LGBMRegressor(
                          num_leaves=best_hyper[best_hyper.subtask == subtask].iloc[0,]['num_leaves'],
                          learning_rate=best_hyper[best_hyper.subtask == subtask].iloc[0,]['learning_rate'],
                          n_estimators=best_hyper[best_hyper.subtask == subtask].iloc[0,]['n_estimators'],
                          min_split_gain=best_hyper[best_hyper.subtask == subtask].iloc[0,]['min_split_gain'],
                          reg_alpha=best_hyper[best_hyper.subtask == subtask].iloc[0,]['reg_alpha'],
                          reg_lambda=best_hyper[best_hyper.subtask == subtask].iloc[0,]['reg_lambda'],
                          feature_fraction=best_hyper[best_hyper.subtask == subtask].iloc[0,]['feature_fraction'],
                          bagging_fraction=best_hyper[best_hyper.subtask == subtask].iloc[0,]['bagging_fraction'],
                          bagging_freq=best_hyper[best_hyper.subtask == subtask].iloc[0,]['bagging_freq'],
                          min_child_samples=best_hyper[best_hyper.subtask == subtask].iloc[0,]['min_child_samples'],
        n_jobs=6, random_state=1, seed=1)

    model.fit(data_tr_x, data_tr_y, eval_metric='auc' if task_type == 'cla' else 'rmse',
              eval_set=[(data_va_x, data_va_y)],
              early_stopping_rounds=patience, verbose=False)
    num_of_compounds = sub_dataset.shape[0]
    import pickle
    pickle.dump(model, open("./model/LGB_"+str(split)+".pkl", "wb"))
    if task_type == 'cla':
        # training set
        tr_pred = model.predict_proba(data_tr_x)
        tr_results = [split, dataset_label, subtask, 'tr', num_fea, num_of_compounds,
                      data_tr_y[data_tr_y == 1].shape[0],
                      data_tr_y[data_tr_y == 0].shape[0],
                      data_tr_y[data_tr_y == 0].shape[0] / data_tr_y[data_tr_y == 1].shape[0]]
        tr_results.extend(statistical(data_tr_y, np.argmax(tr_pred, axis=1), tr_pred[:, 1]))

        # validation set
        va_pred = model.predict_proba(data_va_x)
        va_results = [split, dataset_label, subtask, 'va', num_fea, num_of_compounds,
                      data_va_y[data_va_y == 1].shape[0],
                      data_va_y[data_va_y == 0].shape[0],
                      data_va_y[data_va_y == 0].shape[0] / data_va_y[data_va_y == 1].shape[0]]
        va_results.extend(statistical(data_va_y, np.argmax(va_pred, axis=1), va_pred[:, 1]))

        # test set
        te_pred = model.predict_proba(data_te_x)
        te_results = [split, dataset_label, subtask, 'te', num_fea, num_of_compounds,
                      data_te_y[data_te_y == 1].shape[0],
                      data_te_y[data_te_y == 0].shape[0],
                      data_te_y[data_te_y == 0].shape[0] / data_te_y[data_te_y == 1].shape[0]]
        te_results.extend(statistical(data_te_y, np.argmax(te_pred, axis=1), te_pred[:, 1]))
    else:
        # training set
        tr_pred = model.predict(data_tr_x)
        tr_results = [split, dataset_label, subtask, 'tr', num_fea, num_of_compounds,
                      np.sqrt(mean_squared_error(data_tr_y, tr_pred)), r2_score(data_tr_y, tr_pred),
                      mean_absolute_error(data_tr_y, tr_pred)]

        # validation set
        va_pred = model.predict(data_va_x)
        va_results = [split, dataset_label, subtask, 'va', num_fea, num_of_compounds,
                      np.sqrt(mean_squared_error(data_va_y, va_pred)), r2_score(data_va_y, va_pred),
                      mean_absolute_error(data_va_y, va_pred)]

        # test set
        te_pred = model.predict(data_te_x)
        te_results = [split, dataset_label, subtask, 'te', num_fea, num_of_compounds,
                      np.sqrt(mean_squared_error(data_te_y, te_pred)), r2_score(data_te_y, te_pred),
                      mean_absolute_error(data_te_y, te_pred)]
    return tr_results, va_results, te_results


for subtask in tasks:
    cols = [subtask]
    cols.extend(dataset.columns[(len(tasks) + 1):])
    # cols.extend(dataset.columns[(617+1):])
    sub_dataset = dataset[cols]

#     # detect the NA in the subtask (y cloumn)
#     rm_index = sub_dataset[subtask][sub_dataset[subtask].isnull()].index
#     sub_dataset.drop(index=rm_index, inplace=True)

#     # remove the features with na
#     sub_dataset = sub_dataset.dropna(axis=1)

#     # *******************
#     # demension reduction
#     # *******************
#     # Removing features with low variance
#     # threshold = 0.05
#     data_fea_var = sub_dataset.iloc[:, 1:].var()
#     del_fea1 = list(data_fea_var[data_fea_var <= 0.05].index)
#     sub_dataset.drop(columns=del_fea1, inplace=True)

#     # pair correlations
#     # threshold = 0.95
#     data_fea_corr = sub_dataset.iloc[:, 1:].corr()
#     del_fea2_col = []
#     del_fea2_ind = []
#     length = data_fea_corr.shape[1]
#     for i in range(length):
#         for j in range(i + 1, length):
#             if abs(data_fea_corr.iloc[i, j]) >= 0.95:
#                 del_fea2_col.append(data_fea_corr.columns[i])
#                 del_fea2_ind.append(data_fea_corr.index[j])
#     sub_dataset.drop(columns=del_fea2_ind, inplace=True)

#     # standardize the features
#     cols_ = list(sub_dataset.columns)[1:]
#     if not ecfp :
#         sub_dataset[cols_] = sub_dataset[cols_].apply(standardize, axis=0)

    # for split in range(1, splits+1):
    pool = multiprocessing.Pool(num_pools)
    res = pool.starmap(best_model_runing, zip(range(1, repetitions + 1)))
    pool.close()
    pool.join()
    for item in res:
        for i in range(3):
            pd_res.append(item[i])
if task_type == 'cla':
    stat_res = pd.DataFrame(pd_res, columns=['split', 'dataset', 'subtask', 'set',
                                             'num_of_retained_feature',
                                             'num_of_compounds', 'postives',
                                             'negtives', 'negtives/postives',
                                             'tn', 'fp', 'fn', 'tp', 'se', 'sp',
                                             'auc_prc', 'acc', 'auc_roc','recall','precision','f1','kappa','mcc'])
else:
    stat_res = pd.DataFrame(pd_res, columns=['split', 'dataset', 'subtask', 'set',
                                             'num_of_retained_feature',
                                             'num_of_compounds', 'rmse', 'r2', 'mae'])
stat_res.to_csv('./model/' + dataset_label + '_LGB_statistical_results_split50.csv', index=0)
# single tasks
if len(tasks) == 1:
    args = {'data_label': dataset_label, 'metric': 'auc_roc' if task_type == 'cla' else 'rmse', 'model': 'LGB'}
    print('{}_{}: the mean {} for the training set is {:.3f} with std {:.3f}'.format(args['data_label'], args['model'],
                                                                                     args['metric'], np.mean(
            stat_res[stat_res['set'] == 'tr'][args['metric']]), np.std(
            stat_res[stat_res['set'] == 'tr'][args['metric']])))
    print(
        '{}_{}: the mean {} for the validation set is {:.3f} with std {:.3f}'.format(args['data_label'], args['model'],
                                                                                     args['metric'], np.mean(
                stat_res[stat_res['set'] == 'va'][args['metric']]), np.std(
                stat_res[stat_res['set'] == 'va'][args['metric']])))
    print('{}_{}: the mean {} for the test set is {:.3f} with std {:.3f}'.format(args['data_label'], args['model'],
                                                                                 args['metric'], np.mean(
            stat_res[stat_res['set'] == 'te'][args['metric']]), np.std(
            stat_res[stat_res['set'] == 'te'][args['metric']])))
# multi-tasks
else:
    args = {'data_label': dataset_label, 'metric': 'auc_roc' if dataset_label != 'muv' else 'auc_prc', 'model': 'LGB'}
    tr_acc = np.zeros(repetitions)
    va_acc = np.zeros(repetitions)
    te_acc = np.zeros(repetitions)
    for subtask in tasks:
        tr = stat_res[stat_res['set'] == 'tr']
        tr_acc = tr_acc + tr[tr['subtask'] == subtask][args['metric']].values

        va = stat_res[stat_res['set'] == 'va']
        va_acc = va_acc + va[va['subtask'] == subtask][args['metric']].values

        te = stat_res[stat_res['set'] == 'te']
        te_acc = te_acc + te[te['subtask'] == subtask][args['metric']].values
    tr_acc = tr_acc / len(tasks)
    va_acc = va_acc / len(tasks)
    te_acc = te_acc / len(tasks)
    print('{}_{}: the mean {} for the training set is {:.3f} with std {:.3f}'.format(args['data_label'], args['model'],
                                                                                     args['metric'], np.mean(tr_acc),
                                                                                     np.std(tr_acc)))
    print(
        '{}_{}: the mean {} for the validation set is {:.3f} with std {:.3f}'.format(args['data_label'], args['model'],
                                                                                     args['metric'], np.mean(va_acc),
                                                                                     np.std(va_acc)))
    print('{}_{}: the mean {} for the test set is {:.3f} with std {:.3f}'.format(args['data_label'], args['model'],
                                                                                 args['metric'], np.mean(te_acc),
                                                                                 np.std(te_acc)))
end = time.time()  # get the end time

random seed used in repetition 9 is 9
random seed used in repetition 5 is 5
random seed used in repetition 1 is 1
random seed used in repetition 8 is 8
random seed used in repetition 10 is 10
random seed used in repetition 2 is 2
random seed used in repetition 3 is 3
random seed used in repetition 6 is 6
random seed used in repetition 7 is 7
random seed used in repetition 4 is 4
[LightGBM] [Warning] bagging_fraction is set=0.8891219965439269, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8891219965439269
[LightGBM] [Warning] feature_fraction is set=0.940506078853688, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.940506078853688
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] bagging_fraction is set=0.8891219965439269, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8891219965439269
[LightGBM] [Warning] feature_fraction is set=0.940506078853688, 

In [7]:
# acc auc_roc recall precision f1 kappa mcc
acc_str = 'acc of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['acc']), 
                np.std(stat_res[stat_res['set'] == 'tr']['acc']),
                np.mean(stat_res[stat_res['set'] == 'va']['acc']), 
                np.std(stat_res[stat_res['set'] == 'va']['acc']),
                np.mean(stat_res[stat_res['set'] == 'te']['acc']), 
                np.std(stat_res[stat_res['set'] == 'te']['acc']),
)
auc_str = 'auc_roc of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['auc_roc']), 
                np.std(stat_res[stat_res['set'] == 'tr']['auc_roc']),
                np.mean(stat_res[stat_res['set'] == 'va']['auc_roc']), 
                np.std(stat_res[stat_res['set'] == 'va']['auc_roc']),
                np.mean(stat_res[stat_res['set'] == 'te']['auc_roc']), 
                np.std(stat_res[stat_res['set'] == 'te']['auc_roc']),
)
recall_str = 'recall of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['recall']), 
                np.std(stat_res[stat_res['set'] == 'tr']['recall']),
                np.mean(stat_res[stat_res['set'] == 'va']['recall']), 
                np.std(stat_res[stat_res['set'] == 'va']['recall']),
                np.mean(stat_res[stat_res['set'] == 'te']['recall']), 
                np.std(stat_res[stat_res['set'] == 'te']['recall']),
)
precision_str = 'precision of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['precision']), 
                np.std(stat_res[stat_res['set'] == 'tr']['precision']),
                np.mean(stat_res[stat_res['set'] == 'va']['precision']), 
                np.std(stat_res[stat_res['set'] == 'va']['precision']),
                np.mean(stat_res[stat_res['set'] == 'te']['precision']), 
                np.std(stat_res[stat_res['set'] == 'te']['precision']),
)
f1_str = 'f1 of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['f1']), 
                np.std(stat_res[stat_res['set'] == 'tr']['f1']),
                np.mean(stat_res[stat_res['set'] == 'va']['f1']), 
                np.std(stat_res[stat_res['set'] == 'va']['f1']),
                np.mean(stat_res[stat_res['set'] == 'te']['f1']), 
                np.std(stat_res[stat_res['set'] == 'te']['f1']),
)
kappa_str = 'kappa of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['kappa']), 
                np.std(stat_res[stat_res['set'] == 'tr']['kappa']),
                np.mean(stat_res[stat_res['set'] == 'va']['kappa']), 
                np.std(stat_res[stat_res['set'] == 'va']['kappa']),
                np.mean(stat_res[stat_res['set'] == 'te']['kappa']), 
                np.std(stat_res[stat_res['set'] == 'te']['kappa']),
)
mcc_str = 'mcc of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['mcc']), 
                np.std(stat_res[stat_res['set'] == 'tr']['mcc']),
                np.mean(stat_res[stat_res['set'] == 'va']['mcc']), 
                np.std(stat_res[stat_res['set'] == 'va']['mcc']),
                np.mean(stat_res[stat_res['set'] == 'te']['mcc']), 
                np.std(stat_res[stat_res['set'] == 'te']['mcc']),
)
print('the elapsed time is:', (end - start)/3600, 'H')

the elapsed time is: 4.401333172586229 H


In [8]:
import pandas as pd
import collections
dict1 = {"Model: LGB ":['acc','auc_roc','recall','precision','f1','kappa','mcc','auc_prc'],
         "Train":[np.mean(stat_res[stat_res['set'] == 'tr']['acc']),np.mean(stat_res[stat_res['set'] == 'tr']['auc_roc']),
                  np.mean(stat_res[stat_res['set'] == 'tr']['recall']),np.mean(stat_res[stat_res['set'] == 'tr']['precision']),
                  np.mean(stat_res[stat_res['set'] == 'tr']['f1']),np.mean(stat_res[stat_res['set'] == 'tr']['kappa']),
                  np.mean(stat_res[stat_res['set'] == 'tr']['mcc']),np.mean(stat_res[stat_res['set'] == 'tr']['auc_prc']),                                     
                 ],
         "Tr_STD":[np.std(stat_res[stat_res['set'] == 'tr']['acc']),np.std(stat_res[stat_res['set'] == 'tr']['auc_roc']),
                  np.std(stat_res[stat_res['set'] == 'tr']['recall']),np.std(stat_res[stat_res['set'] == 'tr']['precision']),
                  np.std(stat_res[stat_res['set'] == 'tr']['f1']),np.std(stat_res[stat_res['set'] == 'tr']['kappa']),
                  np.std(stat_res[stat_res['set'] == 'tr']['mcc']),np.std(stat_res[stat_res['set'] == 'tr']['auc_prc']),],
         "Validation":[np.mean(stat_res[stat_res['set'] == 'va']['acc']),np.mean(stat_res[stat_res['set'] == 'va']['auc_roc']),
                      np.mean(stat_res[stat_res['set'] == 'va']['recall']),np.mean(stat_res[stat_res['set'] == 'va']['precision']),
                      np.mean(stat_res[stat_res['set'] == 'va']['f1']),np.mean(stat_res[stat_res['set'] == 'va']['kappa']),
                      np.mean(stat_res[stat_res['set'] == 'va']['mcc']),np.mean(stat_res[stat_res['set'] == 'va']['auc_prc'])],
         "Va_STD":[np.std(stat_res[stat_res['set'] == 'va']['acc']),np.std(stat_res[stat_res['set'] == 'va']['auc_roc']),
                  np.std(stat_res[stat_res['set'] == 'va']['recall']),np.std(stat_res[stat_res['set'] == 'va']['precision']),
                  np.std(stat_res[stat_res['set'] == 'va']['f1']),np.std(stat_res[stat_res['set'] == 'va']['kappa']),
                  np.std(stat_res[stat_res['set'] == 'va']['mcc']),np.std(stat_res[stat_res['set'] == 'va']['auc_prc'])],
         "Test":[np.mean(stat_res[stat_res['set'] == 'te']['acc']),np.mean(stat_res[stat_res['set'] == 'te']['auc_roc']),
                np.mean(stat_res[stat_res['set'] == 'te']['recall']),np.mean(stat_res[stat_res['set'] == 'te']['precision']),
                np.mean(stat_res[stat_res['set'] == 'te']['f1']),np.mean(stat_res[stat_res['set'] == 'te']['kappa']),
                np.mean(stat_res[stat_res['set'] == 'te']['mcc']),np.mean(stat_res[stat_res['set'] == 'te']['auc_prc'])],
          "Te_STD":[np.std(stat_res[stat_res['set'] == 'te']['acc']),np.std(stat_res[stat_res['set'] == 'te']['auc_roc']),
                   np.std(stat_res[stat_res['set'] == 'te']['recall']),np.std(stat_res[stat_res['set'] == 'te']['precision']),
                   np.std(stat_res[stat_res['set'] == 'te']['f1']),np.std(stat_res[stat_res['set'] == 'te']['kappa']),
                   np.std(stat_res[stat_res['set'] == 'te']['mcc']),np.std(stat_res[stat_res['set'] == 'te']['auc_prc']),]}
dict1 = collections.OrderedDict(dict1)
df = pd.DataFrame(dict1,index = None)
df.to_csv('output/output_lgb.csv',index = False)
df

,Model: LGB,Train,Tr_STD,Validation,Va_STD,Test,Te_STD
0,acc,0.986133,0.005093,0.914182,0.009510,0.913912,0.009408
1,auc_roc,0.999147,0.000688,0.940773,0.009724,0.940179,0.006651
2,recall,0.930817,0.024653,0.638532,0.032669,0.653219,0.023923
3,precision,0.995972,0.002911,0.889060,0.034584,0.873520,0.041693
4,f1,0.962148,0.014290,0.742268,0.022003,0.746924,0.024125
5,kappa,0.953676,0.017368,0.692475,0.025207,0.696345,0.028717
6,mcc,0.954598,0.016731,0.706804,0.023004,0.707411,0.029731
7,auc_prc,0.996949,0.002348,0.858343,0.022135,0.854869,0.014950


In [9]:
pd.set_option ( 'display.width', None)
pd.set_option ( 'display.max_columns', None) #显示全部列
hyper_parameters = best_hyper.iloc[0:1,8:-14].T
hyper_parameters.rename(columns={0:'Values'},inplace=True) 
hyper_parameters

,Values
num_leaves,151.000000
learning_rate,0.114327
n_estimators,96.000000
min_split_gain,0.533397
reg_alpha,1.974412
reg_lambda,4.864200
feature_fraction,0.940506
bagging_fraction,0.889122
bagging_freq,3.000000
min_child_samples,56.000000
